# RAG-Pipeline

Load different chunking strategies and their embeddings with the models text-embedding-ada-002 and text-embedding-large-3. 

## Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from openai import AzureOpenAI
import openai

import credentials

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import math

from pinecone import Pinecone, Index, ServerlessSpec

/opt/homebrew/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [15]:
deployment_name='gpt-4'

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

## Load Embeddings

Here we can load precomputed embeddings stored as pickle files. We created these embeddigs in a previous notebook (`chunking_and_embeddings.ipynb`) using different chunking strategies and embedding models (`text-embedding-ada-002` and `text-embedding-large-3`). This ensures consistency across experiments.

In this specific notebook we will load and process the embeddings with recursive character chunking with 1000 chunk size, 0 overlap and the text-embedding-ada-002 model.

In [3]:
# Load embeddings from a pickle file
with open('../embeddings/recursive_500_chunksize_50_overlap_text-embedding-3-large.pkl', 'rb') as f:
    embeddings = pickle.load(f)

**Debugging:** We validate the loaded embeddings by checking for missing or invalid entries. This step ensures we handle API errors or processing failures that may have occurred during embedding generation.

In [4]:
# Filter out rows with None embeddings
valid_embeddings = embeddings[embeddings['embeddings'].notna()]

# Debug: Check how many rows remain
print(f"Total valid embeddings: {len(valid_embeddings)}")
print(f"Total invalid embeddings (None): {len(embeddings) - len(valid_embeddings)}")

Total valid embeddings: 108312
Total invalid embeddings (None): 0


We remove invalid embeddings (rows with None values) to ensure only complete data is processed further. This helps maintain the quality of our pipeline and avoids errors downstream.

## Pinecone Setup and Embedding Management

### Initializing Pinecone Vector Database

We use Pinecone to store and manage our embeddings for retrieval tasks. Pinecone provides a scalable and fast vector database optimized for similarity search.

- Setup Pinecone: We initialize the Pinecone client and check if the required index exists.
- Define Index: If the index does not exist, we create it with the appropriate dimensions to match the embedding model. This ensures compatibility with the stored embeddings.
- Connect to Index: If the index already exists, we directly connect to it, avoiding redundant creation steps.

In [5]:
def initialize_pinecone_index(api_key, index_name, dimension=None, create_if_not_exists=True, cloud="aws", region="us-east-1"):
    """
    Initializes or connects to a Pinecone index.
    
    Args:
        api_key (str): API key for Pinecone.
        index_name (str): The name of the index to connect to or create.
        dimension (int, optional): The dimension of the index embeddings (required if creating an index).
        create_if_not_exists (bool): Whether to create the index if it does not exist. Defaults to True.
        cloud (str): Cloud provider for the index. Defaults to "aws".
        region (str): Region for the index. Defaults to "us-east-1".

    Returns:
        tuple: (Index, bool) where Index is a connected Pinecone Index object, and bool indicates if the index is newly created.
    """
    # Initialize Pinecone client
    pc = Pinecone(api_key=api_key)
    existing_indexes = [index.name for index in pc.list_indexes()]
    is_new_index = False

    if index_name not in existing_indexes:
        if create_if_not_exists:
            if dimension is None:
                raise ValueError("Dimension must be specified when creating a new index.")
            # Create the index
            pc.create_index(
                name=index_name,
                dimension=dimension,
                spec=ServerlessSpec(cloud=cloud, region=region)
            )
            print(f"Index '{index_name}' created.")
            is_new_index = True
        else:
            raise ValueError(f"Index '{index_name}' does not exist and create_if_not_exists is set to False.")
    else:
        print(f"Connecting to existing index '{index_name}'.")

    # Connect to the index
    return pc.Index(index_name), is_new_index

### Upserting Embeddings into Pinecone

After verifying the index status, we decide whether to proceed with the embedding upsertion process:

- Embedding Preparation: We filter valid embeddings and prepare them for upsertion. Each record includes:
    - ID: A unique identifier for the chunk

    - Values: The embedding vector
    
    - Metadata: The original text content for reference and contextual retrieval

- Conditional Upsertion: If the index is newly created, embeddings are upserted in batches. This prevents unnecessary upsertion for existing indexes.

- Batching and Progress Logging: To handle large datasets efficiently and avoid rate-limit issues and help manage memory usage, embeddings are upserted in defined batch sizes, with periodic progress logs to track the process.

In [6]:
def upsert_embeddings(pinecone_index, embeddings, is_new_index, batch_size=100):
    """
    Upserts embeddings into the Pinecone index if it was newly created.

    Args:
        pinecone_index (Index): Connected Pinecone Index object.
        embeddings (pd.DataFrame): DataFrame containing embeddings and metadata.
        is_new_index (bool): Whether the index was newly created.
        batch_size (int): Number of records per upsert batch.
    """
    if not is_new_index:
        print("Index already exists. Skipping upsert.")
        return

    # Prepare records for upserting
    records = []
    for idx, row in embeddings.iterrows():
        doc_id = str(row['index'] if 'index' in embeddings.columns else idx)
        embedding = row['embeddings']
        original_text = row['content_chunks']

        # Validate embedding is a list of floats
        if isinstance(embedding, list) and all(isinstance(value, float) for value in embedding):
            records.append({
                "id": doc_id,
                "values": embedding,
                "metadata": {"text": original_text}
            })

    # Upsert records in batches
    for i in range(0, len(records), batch_size):
        batch = records[i:i + batch_size]
        try:
            pinecone_index.upsert(vectors=batch)
            # Print progress every 50 batches
            if (i // batch_size + 1) % 50 == 0:
                print(f"Upserted batch {i // batch_size + 1}/{(len(records) + batch_size - 1) // batch_size}")
        except Exception as e:
            print(f"Error upserting batch {i // batch_size + 1}: {e}")

This approach optimizes resource utilization and maintains the integrity of the Pinecone index.

In [7]:
# Example Usage
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = "500-chunksize-50-overlap-new-2"
dimension = 1536  # Dimension of the embeddings
valid_embeddings = embeddings[embeddings['embeddings'].notna()]  # Filter valid embeddings

pinecone_index, is_new_index = initialize_pinecone_index(
    api_key=pinecone_api_key,
    index_name=index_name,
    dimension=dimension,
    create_if_not_exists=True
)

upsert_embeddings(
    pinecone_index=pinecone_index,
    embeddings=valid_embeddings,
    is_new_index=is_new_index,
    batch_size=100
)

Connecting to existing index '500-chunksize-50-overlap-new-2'.
Index already exists. Skipping upsert.


## Query Processing (Retriever Module)

The Query Processing module is responsible for embedding user queries, searching the vector database (Pinecone) for the most relevant chunks of context and returning the top-k results. This module forms the backbone of our RAG chain by ensuring that the generator module has high-quality, contextually relevant information.

`generate_embeddings`:
- Generates an embedding vector for a given query using the specified embedding model (same model as the text embedding)
- This vector is used for similarity search in the vector database

`retrieve_relevant_chunks`:
- Accepts a user query, embeds it using the generate_embeddings function and queries the Pinecone vector database to retrieve the most similar chunks.
- Returns a ranked list of chunks based on similarity scores

In [8]:
def generate_embeddings(text, embedding_model):
    # Generate embeddings for a given text using the specified embedding model.
    response = client.embeddings.create(input=[text], model=embedding_model)
    return response.data[0].embedding

def rerank_results(retrieved_chunks, additional_criteria=None):
    """
    Reranks the retrieved chunks based on additional criteria.

    Args:
        retrieved_chunks (list): List of retrieved chunks containing metadata and similarity scores.
        additional_criteria (function, optional): Function that can be used to compute additional scores for reranking.

    Returns:
        list: Reranked list of retrieved chunks.
    """

    def custom_ranking(chunk):
        relevance_score = chunk.metadata.get('relevance_score', 0)  # Default to 0 if no relevance score is available
        return chunk.score + relevance_score  # Combine the scores to create a new ranking score

    # Sort by the custom ranking score in descending order
    reranked_chunks = sorted(retrieved_chunks.matches, key=custom_ranking, reverse=True)
    return reranked_chunks

def retrieve_relevant_chunks(query, embedding_model, top_k=5):
    # Embed the query using the embedding function
    query_embedding = generate_embeddings(query, embedding_model)

    # Retrieve similar documents from Pinecone
    results = pinecone_index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=True,
        include_metadata=True
    )
    
    # Rerank the results based on custom criteria
    reranked_results = rerank_results(results, additional_criteria=None)
    return reranked_results

The current reranking strategy is relatively simple but provides a good starting point for improving the relevance of the retrieved documents. By combining similarity scores with a custom relevance score, the reranking process adjusts the order of results to ensure the most contextually relevant documents are presented first.

If more time and resources were available, the reranking strategy could be significantly enhanced by applying more sophisticated methods such as Learning to Rank, user intent analysis, temporal relevance, or feedback-based adjustments. These strategies would allow for even more personalized and accurate search results, ultimately improving the overall user experience.

### Example Query and Retrieval

The following example demonstrates how to use the `retrieve_relevant_chunks` function to retrieve context for a given query. We defined our own question to test our RAG chain up to this point and used it as a query to retrieve relevant chunks from the Pinecone index.

In [9]:
query = "What did Qatar Petroleum mention what will happen in Phase 1 of the LNG expansion?"
embedding_model = "text-embedding-3-large"

# Retrieve and rerank the results
relevant_chunks = retrieve_relevant_chunks(query, embedding_model, top_k=5)

# Loop through each reranked chunk and print its details
for match in relevant_chunks:
    chunk_id = match.id
    text_content = match.metadata.get('text', '')[:300]
    score = match.score

    print(f"Chunk ID: {chunk_id}")
    print(f"{text_content}...")
    print(f"Similarity Score: {score:.2f}")
    print("-" * 80)

Chunk ID: 60015
would detail emissions generated to produce and deliver the cargoes. The cargoes delivered would not be offset with carbon emissions credits, but they would include certificates detailing the amount of emissions released between the wellhead and import terminal.', 'QP now is working to increase LNG ...
Similarity Score: 0.71
--------------------------------------------------------------------------------
Chunk ID: 3
the Siraj solar power project next year ( EIF Jan.22'20). Until this month, there had been little news about Phase 2 of Qatar's massive LNG expansion. But McDermott International said last week that it had been awarded the front-end engineering and design contract for five offshore wellhead platform...
Similarity Score: 0.71
--------------------------------------------------------------------------------
Chunk ID: 1
facilities by more than 75% and has raised its carbon capture and storage ambitions from 5 million tons/yr to 7 million tons/yr by 2027. About 2

In the output above we can see the Chunk ID, their corresponding text and the calculated cosine similarity score.

## Generate Answers (Generator Module)

We can now implement the Generator Module to generate answers based on the retrieved chunks using Azure OpenAI's GPT-4. In this step, we will create a function that takes a user query and the retrieved chunks, composes a relevant context from those chunks, and then uses GPT-4 to generate an answer based on this context.

- **`model`**: Specifies the model, in our case GPT-4, for generating responses.
- **`max_tokens`**: Limits response length.
- **`temperature`**: Controls randomness; lower = focused, higher = creative.
- **`top_p`**: Limits choices to most likely words for coherent output.
- **`top_k`**: Restricts to top-k choices, narrowing token selection.
- **`stop`**: Defines where the model should stop generating for clean output.

In [10]:
prompt = """
    You are a highly knowledgeable AI assistant specializing in providing accurate and contextually relevant answers. 
    Use the context provided below to answer the user's query as thoroughly and concisely as possible. 
    If the context does not contain sufficient information to answer the query, say so explicitly.
    Do not include any information not supported by the context.
"""

In [19]:
def generate_answer(query, chunks, max_tokens=150, temperature=0.1):
    """
    Generate an answer to the given query using retrieved chunks and Azure OpenAI GPT-4.

    Args:
        query (str): The user query.
        chunks (list): Retrieved chunks containing context.

    Returns:
        str: The generated answer.
    """
    # Compose the context from the retrieved chunks, handling potential missing metadata
    context = " ".join(chunk.get('metadata', {}).get('text', '') for chunk in chunks)

    # Ensure the context isn't empty
    if not context.strip():
        return "The provided context does not contain sufficient information to answer the query."
   
    # Generate the answer using Azure OpenAI GPT-4
    response = client.chat.completions.create(
        messages = [
            {"role": "system", "content": prompt},
            {"role": "system", "content": context},
            {"role": "user", "content": query}
        ],
        model = deployment_name,
        max_tokens = max_tokens,
        temperature = temperature,      # Lower temperature for concise and deterministic answers
        stop = ["."],                   # Optional stop sequence for clean output
    )
    answer = response.choices[0].message.content.strip()

    # Add period at the end of the answer if it doesn't exist
    if answer and answer[-1] not in ['.', '?', '!']:
        answer += "."

    return answer

In [20]:
max_tokens = 150
temperature = 0.1

# Generate answer based on retrieved chunks
generated_answer = generate_answer(query, relevant_chunks, max_tokens, temperature)
print("Generated Answer:\n", generated_answer)

Generated Answer:
 Qatar Petroleum (QP) mentioned that Phase 1 of the LNG expansion, also known as the North Field East project, will contribute to their carbon capture goal.


In [21]:
max_tokens = 150
temperature = 0.9

# Generate answer based on retrieved chunks
generated_answer = generate_answer(query, relevant_chunks, max_tokens, temperature)
print("Generated Answer:\n", generated_answer)

Generated Answer:
 In Phase 1 of the LNG expansion, also known as the North Field East project, Qatar Petroleum (QP) plans to construct facilities that will increase Qatar's LNG capacity by 32 million tons per year.


In [22]:
max_tokens = 1000
temperature = 0.9

# Generate answer based on retrieved chunks
generated_answer = generate_answer(query, relevant_chunks, max_tokens, temperature)
print("Generated Answer:\n", generated_answer)

Generated Answer:
 In Phase 1 of the LNG expansion, also known as the North Field East project, Qatar Petroleum (QP) plans to raise its carbon capture storage from the current capacity to 7 million tons per year by 2027.


### Observations on Answer Generation

In the examples provided, we can observe the following influences of the parameters `temperature` and `max_tokens` on the generated answers:

1. **Influence of Temperature**:
   - At a **lower temperature (0.1)**, the model generates concise and deterministic answers. These responses are focused, consistent and more factual.
   
   - At a **higher temperature (0.9)**, the answers become more creative and varied in phrasing, while still staying accurate to the context. This can be seen in the example above where there is stylistic differences in the responses (a bit more complex to read).

2. **Influence of Max Tokens**:
   - Increasing `max_tokens` to a high value (for example 1000) does not significantly change the answer length for our example questions. The model answers the question efficiently using fewer tokens.
   
   - A higher `max_tokens` ensures that longer contexts or more complex queries can be addressed without truncation, but for simple queries (like our example), the effect is minimal since the required answer fits well within the default limit (for example 150-200 tokens).

These results demonstrate that while `temperature` significantly affects the tone and variability of responses, the `max_tokens` parameter acts as a safeguard to allow the model to generate longer answers when needed but does not force unnecessary verbosity.

## Generating Contexts and Answers for Evaluation Dataset

In this section, we utilize the evaluation dataset to retrieve relevant contexts and generate answers for each query. This allows us to prepare the necessary data for subsequent evaluation of retrieval and generation performance.

We have `data_eval`, which contains the following fields `example_id`, `question_id`, `question`, `relevant_text`, `answer` and `article_url`. Each row represents an evaluation example, with the `question` to be queried in our pipeline, the `relevant_text` providing context for manual verification, and `answer` as the ground-truth answer to compare against.

In [ ]:
data_eval = pd.read_csv('../data/evaluation/cleantech_rag_evaluation_data_2024-09-20.csv', delimiter=';')
data_eval.head()

,example_id,question_id,question,relevant_text,answer,article_url
0,1,1,What is the innovation behind Leclanché's new ...,Leclanché said it has developed an environment...,Leclanché's innovation is using a water-based ...,https://www.sgvoice.net/strategy/technology/23...
1,2,2,What is the EU’s Green Deal Industrial Plan?,The Green Deal Industrial Plan is a bid by the...,The EU’s Green Deal Industrial Plan aims to en...,https://www.sgvoice.net/policy/25396/eu-seeks-...
2,3,2,What is the EU’s Green Deal Industrial Plan?,The European counterpart to the US Inflation R...,The EU’s Green Deal Industrial Plan aims to en...,https://www.pv-magazine.com/2023/02/02/europea...
3,4,3,What are the four focus areas of the EU's Gree...,The new plan is fundamentally focused on four ...,The four focus areas of the EU's Green Deal In...,https://www.sgvoice.net/policy/25396/eu-seeks-...
4,5,4,When did the cooperation between GM and Honda ...,What caught our eye was a new hookup between G...,July 2013,https://cleantechnica.com/2023/05/08/general-m...


### Pipeline Setup

A custom RAGPipeline class is instantiated with two core modules:

- Retriever: Fetches relevant chunks using Pinecone based on the query embedding.
- Generator: Generates an answer using the retrieved chunks and GPT-4.

In [ ]:
class RAGPipeline:
    def __init__(self, retriever, generator):
        self.retriever = retriever
        self.generator = generator

    def retrieve_relevant_chunks(self, query, embedding_model, top_k=5):
        return self.retriever(query, embedding_model, top_k)

    def generate_answer(self, query, retrieved_chunks):
        return self.generator(query, retrieved_chunks)

# Instantiate the pipeline
pipeline = RAGPipeline(
    retriever=retrieve_relevant_chunks,
    generator=generate_answer
)

### Scoring

- BLEU and ROUGE scores are calculated to measure textual overlap between the ground truth and the generated answer.
- Cosine similarity between embeddings of the ground truth and generated answer is computed to assess semantic similarity.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def calculate_bleu_scores(reference, hypothesis):
    smoothing_function = SmoothingFunction().method1

    scores = {}
    for n in range(1, 5):  # BLEU-1 to BLEU-4
        weights = tuple((1 / n) for _ in range(n)) + (0,) * (4 - n)
        scores[f"bleu_{n}"] = sentence_bleu(
            [reference.split()],
            hypothesis.split(),
            weights=weights,
            smoothing_function=smoothing_function
        )
    return scores

def calculate_rouge_scores(reference, hypothesis):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge_scores = rouge.score(reference, hypothesis)
    rouge1 = rouge_scores['rouge1'].fmeasure
    rouge2 = rouge_scores['rouge2'].fmeasure
    rougeL = rouge_scores['rougeL'].fmeasure
    return {"rouge1": rouge1, "rouge2": rouge2, "rougeL": rougeL}

### Retrieving Contexts and Generating Answers

For each query in the evaluation dataset:

- Retrieve the most relevant contexts using Pinecone.
- Generate an answer based on the retrieved contexts using the generate_answer function.

In [ ]:
def evaluate_example(query, true_answer, pipeline, embedding_model):
    # Retrieve relevant chunks and generate an answer
    relevant_chunks = pipeline.retrieve_relevant_chunks(query, embedding_model, top_k=5)
    generated_answer = pipeline.generate_answer(query, relevant_chunks)

    # Cosine Similarity
    true_embedding = np.array(generate_embeddings(true_answer, embedding_model)).reshape(1, -1)
    generated_embedding = np.array(generate_embeddings(generated_answer, embedding_model)).reshape(1, -1)
    cosine_sim = cosine_similarity(true_embedding, generated_embedding)[0][0]

    # BLEU and ROUGE Scores
    bleu_scores = calculate_bleu_scores(true_answer, generated_answer)
    rouge_scores = calculate_rouge_scores(true_answer, generated_answer)

    # relevant chunks to list
    relevant_chunks_list = [chunk.get('metadata', {}).get('text', '') for chunk in relevant_chunks['matches']]

    return {
        "question": query,
        "ground_truth": true_answer,
        "answer": generated_answer,
        "contexts": relevant_chunks_list,
        **bleu_scores,
        **rouge_scores,
        "cosine_similarity": cosine_sim
    }

def evaluate_pipeline(data_eval, pipeline, embedding_model):
    results = []
    for _, row in tqdm(data_eval.iterrows(), total=len(data_eval), desc="Retrieving Contexts and Generating Answers:"):
        query = row['question']
        true_answer = row['answer']
        
        # Evaluate this example
        example_results = evaluate_example(query, true_answer, pipeline, embedding_model)
        results.append(example_results)
    
    return pd.DataFrame(results)

evaluation_results = evaluate_pipeline(data_eval, pipeline, embedding_model)

Evaluating RAG Pipeline: 100%|██████████| 23/23 [04:05<00:00, 10.66s/it]


Now that we have retrieved our top-k chunks and generated answers for each query, we can save this data and use it later in a different notebook to evaluate the performance of our RAG pipeline using the evaluation dataset.

In [ ]:
# save dataset to csv
evaluation_results.to_csv('../data/recursive_2500_chunksize_300_overlap_text_embedding_3_large.csv', index=False)